# Oregon Air Quality Dashboard (PM2.5)

This notebook shows county-level PM2.5 and Ozone in Oregon over time, using:
- AQS historical data (2020–2024)
- Oregon county boundaries (GeoJSON)
- An interactive date slider

Use the slider below to explore how PM2.5 and Ozone vary by county on different days.


In [ ]:
from pathlib import Path
import json

import pandas as pd
import plotly.express as px

import plotly.io as pio

import ipywidgets as widgets
from IPython.display import display, clear_output

pio.renderers.default = "notebook"  # try this first
# If that still shows nothing, later try: pio.renderers.default = "iframe_connected"

# Base data directory (relative to this notebook in /notebooks)
DATA_DIR = Path("../data").resolve()

# Load Oregon-only counties GeoJSON
with open(DATA_DIR / "oregon_counties.geojson", "r") as f:
    oregon_geojson = json.load(f)

In [ ]:
# Adjust this filename to whatever you used for pm25 daily data
pm25_path = DATA_DIR / "aqs_daily_pm25_20200101_20241231.parquet"

df_pm25 = pd.read_parquet(pm25_path)

# Adjust this filename to whatever you used for ozone daily data
ozone_path = DATA_DIR / "aqs_daily_ozone_20200101_20241231.parquet"

df_ozone = pd.read_parquet(ozone_path)

In [ ]:
geo_counties = sorted({f["properties"]["NAME"] for f in oregon_geojson["features"]})

In [ ]:
df_geo = pd.DataFrame({"county": geo_counties})

df_map = df_geo.merge(
    df_pm25.rename(columns={"county_clean": "county"}),
    how="left",
    on="county"
)


In [ ]:
date_col = "date_local"
county_col = "county"
value_col = "arithmetic_mean"  # or whatever your PM2.5 value column is


In [ ]:
def compute_pm25_by_county(df, target_date):
    """Return a df with columns ['county', 'pm25_mean'] for one date."""
    
    # Filter to one date
    subset = df[df[date_col] == target_date]

    # Aggregate
    pm25_latest = (
        subset
        .groupby(county_col, as_index=False)
        .agg(pm25_mean=(value_col, "mean"))
    )

    # Build full county list
    df_geo = pd.DataFrame({"county": geo_counties})

    # Merge (left join)
    df_map = df_geo.merge(
        pm25_latest.rename(columns={county_col: "county"}),
        on="county",
        how="left"
    )

    return df_map   


In [ ]:
def summarize_pm25_for_date(df, target_date):
    """
    Compute simple summary stats for PM2.5 on one date.
    Returns a dict with statewide_mean, worst_county, worst_value, best_county, best_value.
    """
    df_map = compute_pm25_by_county(df, target_date)

    df_nonnull = df_map[df_map["pm25_mean"].notna()]

    if df_nonnull.empty:
        return {
            "statewide_mean": None,
            "worst_county": None,
            "worst_value": None,
            "best_county": None,
            "best_value": None,
        }

    statewide_mean = df_nonnull["pm25_mean"].mean()

    worst_row = df_nonnull.loc[df_nonnull["pm25_mean"].idxmax()]
    best_row = df_nonnull.loc[df_nonnull["pm25_mean"].idxmin()]

    return {
        "statewide_mean": statewide_mean,
        "worst_county": worst_row["county"],
        "worst_value": worst_row["pm25_mean"],
        "best_county": best_row["county"],
        "best_value": best_row["pm25_mean"],
    }


In [ ]:
def plot_pm25_map_for_date(df, target_date):
    # Compute the county-level data for the chosen date
    df_map = compute_pm25_by_county(df, target_date)

    # Build the map
    fig = px.choropleth_mapbox(
        df_map,
        geojson=oregon_geojson,
        locations="county",
        featureidkey="properties.NAME",
        color="pm25_mean",
        color_continuous_scale="Viridis",
        range_color=(df_map["pm25_mean"].min(), df_map["pm25_mean"].max()),
        mapbox_style="carto-positron",
        zoom=4.4,
        center={"lat": 44.0, "lon": -120.5},
        opacity=0.75,
    )

    fig.update_layout(
        title=f"Oregon PM2.5 Mean by County on {target_date}",
        margin={"r": 0, "t": 40, "l": 0, "b": 0},
    )

    return fig


In [ ]:
oz_date_col = "date_local"
oz_county_col = "county"
oz_value_col = "arithmetic_mean"  # change if needed after you see columns

In [ ]:
def compute_ozone_by_county(df, target_date):
    """Return a df with columns ['county', 'ozone_mean'] for one date."""
    subset = df[df[oz_date_col] == target_date]

    ozone_latest = (
        subset
        .groupby(oz_county_col, as_index=False)
        .agg(ozone_mean=(oz_value_col, "mean"))
    )

    df_geo = pd.DataFrame({"county": geo_counties})

    df_map = df_geo.merge(
        ozone_latest.rename(columns={oz_county_col: "county"}),
        on="county",
        how="left"
    )

    return df_map


In [ ]:
def plot_ozone_map_for_date(df, target_date):
    df_map = compute_ozone_by_county(df, target_date)

    fig = px.choropleth_mapbox(
        df_map,
        geojson=oregon_geojson,
        locations="county",
        featureidkey="properties.NAME",
        color="ozone_mean",
        color_continuous_scale="Viridis",
        range_color=(df_map["ozone_mean"].min(), df_map["ozone_mean"].max()),
        mapbox_style="carto-positron",
        zoom=4.4,
        center={"lat": 44.0, "lon": -120.5},
        opacity=0.75,
    )

    fig.update_layout(
        title=f"Oregon Ozone Mean by County on {target_date}",
        margin={"r": 0, "t": 40, "l": 0, "b": 0},
    )

    return fig


In [ ]:
def compute_ozone_by_county(df, target_date):
    """Return a df with columns ['county', 'ozone_mean'] for one date."""
    subset = df[df[oz_date_col] == target_date]

    ozone_latest = (
        subset
        .groupby(oz_county_col, as_index=False)
        .agg(ozone_mean=(oz_value_col, "mean"))
    )

    df_geo = pd.DataFrame({"county": geo_counties})

    df_map = df_geo.merge(
        ozone_latest.rename(columns={oz_county_col: "county"}),
        on="county",
        how="left"
    )

    return df_map


def summarize_ozone_for_date(df, target_date):
    """Summary stats for ozone on one date."""
    df_map = compute_ozone_by_county(df, target_date)
    df_nonnull = df_map[df_map["ozone_mean"].notna()]

    if df_nonnull.empty:
        return {
            "statewide_mean": None,
            "worst_county": None,
            "worst_value": None,
            "best_county": None,
            "best_value": None,
        }

    statewide_mean = df_nonnull["ozone_mean"].mean()
    worst_row = df_nonnull.loc[df_nonnull["ozone_mean"].idxmax()]
    best_row = df_nonnull.loc[df_nonnull["ozone_mean"].idxmin()]

    return {
        "statewide_mean": statewide_mean,
        "worst_county": worst_row["county"],
        "worst_value": worst_row["ozone_mean"],
        "best_value": best_row["ozone_mean"],
        "best_county": best_row["county"],
    }


In [ ]:
def plot_ozone_map_for_date(df, target_date):
    df_map = compute_ozone_by_county(df, target_date)

    # Guard against all-NaN days
    if df_map["ozone_mean"].notna().any():
        vmin = df_map["ozone_mean"].min()
        vmax = df_map["ozone_mean"].max()
    else:
        vmin, vmax = 0, 1  # dummy range; map will just look blank

    fig = px.choropleth_mapbox(
        df_map,
        geojson=oregon_geojson,
        locations="county",
        featureidkey="properties.NAME",
        color="ozone_mean",
        color_continuous_scale="Viridis",
        range_color=(vmin, vmax),
        mapbox_style="carto-positron",
        zoom=4.4,
        center={"lat": 44.0, "lon": -120.5},
        opacity=0.75,
    )

    fig.update_layout(
        title=f"Oregon Ozone Mean by County on {target_date}",
        margin={"r": 0, "t": 40, "l": 0, "b": 0},
    )

    return fig


In [ ]:
available_dates = sorted(df_pm25[date_col].unique())

pollutant_dropdown = widgets.Dropdown(
    options=["PM2.5", "Ozone"],
    value="PM2.5",
    description="Pollutant:",
)

date_selector = widgets.SelectionSlider(
    options=available_dates,
    value=available_dates[-1],
    description="Date:",
    continuous_update=False,
    layout=widgets.Layout(width="90%")
)

output_map = widgets.Output()
output_stats = widgets.Output()
output_line = widgets.Output()

date_col = "date_local"  # if not already defined

def make_statewide_mean_figure(df: pd.DataFrame, title_prefix: str, anchor_date, n: int = 90):
    """
    Build a line chart of statewide daily mean up to a given anchor date.
    Shows the last n days up to and including anchor_date.
    """
    if df.empty:
        return None

    if date_col not in df.columns or "arithmetic_mean" not in df.columns:
        return None

    # Filter to dates up to anchor_date
    daily = (
        df[df[date_col] <= anchor_date]
        .groupby(date_col, as_index=False)["arithmetic_mean"]
        .mean()
        .rename(columns={"arithmetic_mean": "statewide_daily_mean"})
        .sort_values(date_col)
    )

    if daily.empty:
        return None

    if len(daily) > n:
        daily = daily.tail(n)

    # Optional: make a nice title that includes the anchor date
    title = f"{title_prefix} (through {anchor_date})"

    fig = px.line(
        daily,
        x=date_col,
        y="statewide_daily_mean",
        title=title,
        markers=True,
    )
    fig.update_layout(height=360, margin=dict(l=10, r=10, t=40, b=10))

    # Optional: emphasize the anchor_date point if present
    if anchor_date in daily[date_col].values:
        fig.add_vline(x=anchor_date, line_dash="dash", line_width=1)

    return fig


def update_dashboard(change=None):
    selected_date = date_selector.value
    selected_pollutant = pollutant_dropdown.value

    # update map
    with output_map:
        clear_output(wait=True)
        if selected_pollutant == "PM2.5":
            fig = plot_pm25_map_for_date(df_pm25, selected_date)
        else:
            fig = plot_ozone_map_for_date(df_ozone, selected_date)
        fig.show()

    # update stats
    with output_stats:
        clear_output(wait=True)
        if selected_pollutant == "PM2.5":
            summary = summarize_pm25_for_date(df_pm25, selected_date)
            metric_name = "PM2.5"
        else:
            summary = summarize_ozone_for_date(df_ozone, selected_date)
            metric_name = "Ozone"

        if summary["statewide_mean"] is None:
            print(f"No {metric_name} data available for this date.")
        else:
            print(f"Date: {selected_date}")
            print(f"Pollutant: {metric_name}")
            print(f"Statewide mean (counties with data): {summary['statewide_mean']:.2f}")
            print(f"Worst county: {summary['worst_county']} ({summary['worst_value']:.2f})")
            print(f"Best county: {summary['best_county']} ({summary['best_value']:.2f})")

    # Line chart: last 90 days up to selected_date
    with output_line:
        clear_output(wait=True)
        if selected_pollutant == "PM2.5":
            fig_line = make_statewide_mean_figure(
                df_pm25,
                "Oregon PM2.5 statewide mean (last 90 days)",
                anchor_date=selected_date,
                n=90,
            )
        else:
            fig_line = make_statewide_mean_figure(
                df_ozone,
                "Oregon Ozone statewide mean (last 90 days)",
                anchor_date=selected_date,
                n=90,
            )

        if fig_line is None:
            print("No data available for line chart.")
        else:
            fig_line.show()

pollutant_dropdown.observe(update_dashboard, names="value")
date_selector.observe(update_dashboard, names="value")

display(pollutant_dropdown, date_selector, output_map, output_stats, output_line)

update_dashboard()
